In [36]:
%load_ext Cython
%time
%cython

ERROR:root:Line magic function `%cython` not found (But cell magic `%%cython` exists, did you mean that instead?).


The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython
CPU times: user 5 µs, sys: 4 µs, total: 9 µs
Wall time: 13.1 µs


In [180]:
#import needed packages
from gensim.models import doc2vec

import re
import pandas as pd
import numpy as np
import copy

# nltk processing
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import movie_reviews

In [181]:
import sqlite3
conn = sqlite3.connect('capstone.db')
# tblGame , tblMovie, tblReview, tblTVShow

In [182]:
# load games dataset
games = pd.read_sql_query("select * from tblGame;", conn)
games['gameID'] = games.index + 1
games.head(5)

,name,link,image,developer,genre,rating,rlsDate,summary,gameID
0,Ginger: Beyond the Crystal,/game/playstation-4/ginger-beyond-the-crystal,http://static.metacritic.com/images/products/g...,Drakhar Studio,Action,E,2016-10-25 00:00:00,"A world destroyed by a crystal explosion, a my...",1
1,JoJo's Bizarre Adventure: Eyes of Heaven,/game/playstation-4/jojos-bizarre-adventure-ey...,http://static.metacritic.com/images/products/g...,CyberConnect2,Action,T,2016-06-28 00:00:00,"Featuring a new two-on-two battle system, larg...",2
2,The Crew,/game/playstation-4/the-crew,http://static.metacritic.com/images/products/g...,"Ubisoft Reflections, Ivory Tower",Racing,T,2014-12-02 00:00:00,Strap in for a ride that will find you infiltr...,3
3,Batman: Arkham Knight - A Matter of Family,/game/playstation-4/batman-arkham-knight---a-m...,http://static.metacritic.com/images/products/g...,"Rocksteady Studios, WB Games Montreal",Action Adventure,M,2015-07-14 00:00:00,When The Joker kidnaps Commissioner Gordon and...,4
4,8DAYS,/game/playstation-4/8days,http://static.metacritic.com/images/products/g...,Santa Clara Games,Action,M,2017-02-07 00:00:00,The world has been dominated by dark forces si...,5


In [183]:
# Read in Critic Reviews
reviews = pd.read_sql_query("select * from tblReview;", conn)


In [305]:
# subset for just critic reviews to build a model
subset = reviews[reviews['reviewType']=='c']
subset.head(5)

,gameID,movieID,tvShowID,author,publication,text,score,date,thumbsUp,thumbsDown,reviewType
0,1.0,NaN,NaN,,Areajugones,Ginger: Beyond the Crystal is a very good plat...,69,2016-11-05 00:00:00,0,0,c
1,1.0,NaN,NaN,,Hobby Consolas,"It\s not a bad game at all, but it has technic...",67,2016-11-02 00:00:00,0,0,c
2,1.0,NaN,NaN,,GameGrin,Ginger: Beyond the Crystal tries to revisit th...,60,2016-11-07 00:00:00,0,0,c
3,1.0,NaN,NaN,,GameSpew,It may not offer much of a challenge or much i...,60,2016-11-05 00:00:00,0,0,c
4,1.0,NaN,NaN,,Digitally Downloaded,"It may sound like I’m being harsh on Ginger, b...",50,2016-11-21 00:00:00,0,0,c


In [190]:
# get a unique review corpus
review_docs = subset.groupby(['gameID'])['text'].apply(lambda x: ''.join(x)).reset_index()
review_docs.head(5)

,gameID,text
0,1.0,Ginger: Beyond the Crystal is a very good plat...
1,2.0,JoJo’s Bizarre Adventure: Eyes in Heaven featu...
2,3.0,The Crew does a lot of things right. Not only ...
3,4.0,Playing as Batgirl is not quite as interesting...
4,5.0,The frantic nature of the gameplay coupled wit...


In [302]:
review_docs = pd.merge(games[['gameID']], review_docs, on = "gameID", how = "left")
review_docs.loc[np.any(review_docs.isnull(), axis = 1),'text'] = ""


In [303]:
stopwrds = stopwords.words('english')
# aux function to clean up text
def cleaning_text(sentence):
    sentence = str(sentence)
    sentence = sentence.lower()
    sentence = re.sub('[^\w\s]',' ', sentence)
    sentence = re.sub('_',' ', sentence)
    sentence = re.sub('\d+',' ', sentence)
    cleaned = ' '.join([w for w in sentence.split() if not w in stopwrds])
    cleaned = ' '.join([w for w , pos in pos_tag(cleaned.split()) if (pos == 'NN' or pos=='JJ' or pos=='JJR' or pos=='JJS' )])
    cleaned = ' '.join([w for w in cleaned.split() if not len(w)<=2 ])
    cleaned = cleaned.strip()
    return cleaned

In [304]:
# clean up review docs, add utf8 encoding
review_docs['textClean'] = review_docs.apply(lambda row: cleaning_text(row['text'].encode("utf8")), axis=1)

In [308]:
review_docs.shape

(20416, 3)

In [194]:
# Assumption 0 is gameID, 2 is the text itselx

# define a few functions for doc2vec processing
def split_sentence(sentence):
    words = re.split('\W+', sentence.lower())
    return [word for word in words if word != ""]

# MyDocs reading from a data frame
class MyDocs(object):
    def __iter__(self):
        for i in range(review_docs.shape[0]):
            yield doc2vec.LabeledSentence(words=split_sentence(review_docs.iloc[i,2]), tags=['%s' % review_docs.iloc[i,0]])

In [195]:
review_docs.iloc[0,0]

1.0

In [309]:
# Train the doc2vec model
mydocs = MyDocs()
model = doc2vec.Doc2Vec(mydocs, size = 200, window = 8, min_count = 5, workers = 4)
model.save("review.model")

In [312]:
# testing similar words
print model.most_similar(positive=["cheap", "zelda"], negative=["slow"], topn=5)

[('princess', 0.5255189538002014), ('rerelease', 0.5231113433837891), ('sotn', 0.5113462805747986), ('gunstar', 0.5000026226043701), ('deluxe', 0.4974147379398346)]


In [313]:
# Auxiliary functions for simple recommendation system 

# Calculate cosine similarity between two vecotrs 
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2)) 

# return top_n values from a list
def top_n_index(l,n):
    return sorted(range(len(l)), key=lambda i: l[i])[-(n+1):-1] #-1 to take off the own product from the returned index list

# return a dataframe with top_n products with cosine similarity score
def recommend2(game_index, top_n):
    input_vec = model.docvecs[review_docs.loc[review_docs['gameID']==game_index].index[0]]
    
    #compute similarity array
    sim_array = map(lambda v: cossim(input_vec, v), model.docvecs)
    
    index = top_n_index(sim_array, top_n)
    
    # return list of products 
    top_products = copy.deepcopy(review_docs.iloc[index,][['gameID']])
    top_products['index']= top_products.index
    
     # initialize an empty column for cosine similarity
    top_products['cossim']=0
    
    for i in range(len(index)):
        # note - this results in a bug: top_products['cossim'].loc[index[i],] = sim_array[index[i]]
        top_products.loc[index[i],'cossim'] = sim_array[index[i]]
        
    # merge game name
    top_products['gameID'] = top_products.apply(lambda row: int(row['gameID']),axis = 1)
    top_products = pd.merge(res, games[['gameID','name']], on='gameID')
    
    return top_products

20416

In [342]:
games.loc[games['name'].str.contains('Zelda')]

,name,link,image,developer,genre,rating,rlsDate,summary,gameID
637,The Legend of Zelda: Twilight Princess,/game/gamecube/the-legend-of-zelda-twilight-pr...,http://static.metacritic.com/images/products/g...,Nintendo,Action Adventure,T,2006-12-11 00:00:00,In the next chapter in the Legend of Zelda ser...,638
653,The Legend of Zelda: The Wind Waker,/game/gamecube/the-legend-of-zelda-the-wind-waker,http://static.metacritic.com/images/products/g...,Nintendo,Action Adventure,E,2003-03-24 00:00:00,In this ninth entry in the Legend of Zelda ser...,654
670,The Legend of Zelda Collector's Edition,/game/gamecube/the-legend-of-zelda-collectors-...,http://static.metacritic.com/images/products/g...,Nintendo,Action Adventure,E,2003-11-17 00:00:00,Receive Four Complete Classic Zelda games all ...,671
890,The Legend of Zelda: Phantom Hourglass,/game/ds/the-legend-of-zelda-phantom-hourglass,http://static.metacritic.com/images/products/g...,Nintendo,Action Adventure,E,2007-10-01 00:00:00,[Metacritic\s 2007 DS Game of the Year] Many ...,891
935,The Legend of Zelda: Ocarina of Time / Master ...,/game/gamecube/the-legend-of-zelda-ocarina-of-...,http://static.metacritic.com/images/products/g...,Nintendo,Action Adventure,E,2003-02-28 00:00:00,The Legend of Zelda: Ocarina of Time Master Qu...,936
1109,The Legend of Zelda: Spirit Tracks,/game/ds/the-legend-of-zelda-spirit-tracks,http://static.metacritic.com/images/products/g...,Nintendo,Action Adventure,E10+,2009-12-07 00:00:00,The Legend of Zelda: Spirit Tracks sets hero L...,1110
1446,The Legend of Zelda: Twilight Princess,/game/wii/the-legend-of-zelda-twilight-princess,http://static.metacritic.com/images/products/g...,Nintendo,Action Adventure,T,2006-11-19 00:00:00,When an evil darkness enshrouds the land of Hy...,1447
1535,The Legend of Zelda: Skyward Sword,/game/wii/the-legend-of-zelda-skyward-sword,http://static.metacritic.com/images/products/g...,Nintendo,Action Adventure,E10+,2011-11-20 00:00:00,A Link to the Future - The Legend of Zelda: Sk...,1536
1544,The Legend of Zelda: Four Swords Anniversary E...,/game/ds/the-legend-of-zelda-four-swords-anniv...,http://static.metacritic.com/images/products/g...,GREZZO,Action Adventure,E10+,2011-09-28 00:00:00,The Legend of Zelda: Four Swords is a free DSi...,1545
1882,The Legend of Zelda: Four Swords Adventures,/game/gamecube/the-legend-of-zelda-four-swords...,http://static.metacritic.com/images/products/g...,Nintendo,Action Adventure,E,2004-06-07 00:00:00,The legend continues on the GameCube in an all...,1883


In [318]:
# "Basic recommendation system with doc2vec

# Select gameID of a product you like
gameID = 270
#recommendation
print "You liked this product::: %s" %games.loc[games['gameID']==gameID]['name']
recommend2(gameID,5)

You liked this product::: 269    Monsters Invade: Oz
Name: name, dtype: object


,gameID,index,cossim,name
0,5437.0,5434,0.822358,Magician's Quest: Mysterious Times
1,6766.0,6762,0.825684,Clay's Reverie
2,709.0,708,0.830000,Camping Mama: Outdoor Adventures
3,657.0,656,0.830050,Code Lyoko: Fall of X.A.N.A.
4,3639.0,3637,0.847376,The Munchables


In [94]:
#save a pickle file
import pickle

with open('models/model_critic_only.pickle', 'wb') as handle:
    pickle.dump(model, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [4]:
import pickle
#to load a pickle file
with open('models/model_critic_only.pickle', 'rb') as f:
   model_critic_only = pickle.load(f)   

In [121]:
sum(summary_docs.apply(lambda row: row["textClean"]=="", axis = 1))

472

In [128]:
model_summary.

<function gensim.models.word2vec.log_accuracy>

In [62]:
# get a unique summary corpus
summary_docs = games[['gameID','summary']]
summary_docs.head(5)

,gameID,summary
0,1,"A world destroyed by a crystal explosion, a my..."
1,2,"Featuring a new two-on-two battle system, larg..."
2,3,Strap in for a ride that will find you infiltr...
3,4,When The Joker kidnaps Commissioner Gordon and...
4,5,The world has been dominated by dark forces si...


In [80]:
# clean up summary docs, add utf8 encoding when there is no None Type
summary_docs['textClean'] = summary_docs.apply(lambda row: row['summary'] if row['summary'] is None else cleaning_text(row['summary'].encode("utf8")), axis=1)

/Users/yvonnelau/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [129]:
# define class for summary Doc2vec

# MyDocs reading from a data frame
class MyDocs_summary(object):
    def __iter__(self):
        for i in range(summary_docs.shape[0]):
            yield doc2vec.LabeledSentence(words=split_sentence(summary_docs.iloc[i,2]), tags=['%s' % i])

In [130]:
# Train the doc2vec model
mydocs_summary = MyDocs_summary()
model_summary = doc2vec.Doc2Vec(mydocs_summary, size = 200, window = 8, min_count = 5, workers = 4)
model_summary.save("summary.model")

In [131]:
# testing similar words
print model_summary.most_similar(positive=["cheap", "zelda"], negative=["slow"], topn=5)

[('peter', 0.9551876783370972), ('penultimate', 0.9540131092071533), ('upcoming', 0.9488170146942139), ('author', 0.9409167766571045), ('hobbit', 0.9403274059295654)]


In [152]:
# Auxiliary functions for simple recommendation system 

# Calculate cosine similarity between two vecotrs 
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2)) 

# return top_n values from a list
def top_n_index(l,n):
    return sorted(range(len(l)), key=lambda i: l[i])[-(n+1):-1] #-1 to take off the own product from the returned index list

# return a dataframe with top_n products with cosine similarity score
def recommend2(l,n):
    # get indexes of top_n similar items 
    index = top_n_index(l, n)
    
    # return list of products 
    top_products = copy.deepcopy(summary_docs.iloc[index,][['gameID']])
    top_products['index']= top_products.index
    
    # initialize an empty column for cosine similarity
    top_products['cossim']=0
    
    for i in range(len(index)):
        # note - this results in a bug: top_products['cossim'].loc[index[i],] = sim_array[index[i]]
        top_products.loc[index[i],'cossim'] = l[index[i]]
        
    return top_products   

In [155]:
# "Basic recommendation system with doc2vec

# Select gameID of a product you like
gameID = 836
input_vec = model_summary.docvecs[summary_docs.loc[summary_docs['gameID']==gameID].index[0]]

# Calculated a cossim matrix between input product and model's vectors
sim_array_summary = map(lambda v: cossim(input_vec, v), model_summary.docvecs)

#recommendation
print "You liked this product::: %s" %games.loc[games['gameID']==gameID]['name']
res = recommend2(sim_array_summary,5)
res['gameID'] = res.apply(lambda row: int(row['gameID']),axis = 1)
pd.merge(res, games[['gameID','name']], on='gameID')

You liked this product::: 835    Super Mario Sunshine
Name: name, dtype: object


,gameID,index,cossim,name
0,17831,17830,0.946774,Grim Fandango Remastered
1,113,112,0.956355,Charlie and the Chocolate Factory
2,17301,17300,0.956849,The Curse of Monkey Island
3,6900,6899,0.960252,LEGO Batman: DC Super Heroes
4,18760,18759,0.969277,Oddworld: Abe's Oddysee - New 'n' Tasty


In [143]:
index = top_n_index(sim_array_summary,5)

In [144]:
top_products = copy.deepcopy(summary_docs.iloc[index,][['gameID']])

In [146]:
top_products 

,gameID
11379,11380
16005,16006
7692,7693
6155,6156
1646,1647


In [161]:
len(model_summary.docvecs)

20416

In [162]:
len(model.docvecs)

20138

In [178]:
np.concatenate((model_summary.docvecs[0],model.docvecs[0]), axis = 0)

array([  1.54701900e-02,   8.51636007e-03,  -5.33457845e-03,
        -2.08204053e-02,  -1.56388544e-02,  -1.36512974e-02,
        -3.03671393e-03,  -1.30741065e-03,  -1.52839497e-02,
         9.72414948e-03,  -2.00144071e-02,  -3.30307730e-03,
         1.38337826e-02,  -4.44546901e-03,   3.88827920e-03,
        -7.83279014e-04,  -4.39949473e-03,  -1.81716979e-02,
        -7.27863982e-03,  -7.75696011e-03,  -5.58632938e-03,
        -1.82472803e-02,   1.51122538e-02,   8.63296445e-05,
         2.00901516e-02,   1.14349672e-03,   2.16712244e-03,
        -1.01414118e-02,  -1.23933819e-03,   4.66667069e-03,
        -2.32808907e-02,  -5.01756044e-03,  -2.54788511e-02,
        -3.53118521e-03,  -1.14892712e-02,  -1.87425874e-02,
        -1.48088261e-02,  -2.20603272e-02,   5.83948568e-03,
        -3.14036338e-03,  -9.70700011e-03,   6.79585803e-03,
         1.64701790e-02,   1.30476002e-02,   5.90754999e-03,
         9.86126810e-03,  -1.49589302e-02,   1.58847566e-03,
         1.86211150e-03,

In [164]:
games.shape

(20416, 9)

In [306]:
reviews

,gameID,movieID,tvShowID,author,publication,text,score,date,thumbsUp,thumbsDown,reviewType
0,1.0,NaN,NaN,,Areajugones,Ginger: Beyond the Crystal is a very good plat...,69,2016-11-05 00:00:00,0,0,c
1,1.0,NaN,NaN,,Hobby Consolas,"It\s not a bad game at all, but it has technic...",67,2016-11-02 00:00:00,0,0,c
2,1.0,NaN,NaN,,GameGrin,Ginger: Beyond the Crystal tries to revisit th...,60,2016-11-07 00:00:00,0,0,c
3,1.0,NaN,NaN,,GameSpew,It may not offer much of a challenge or much i...,60,2016-11-05 00:00:00,0,0,c
4,1.0,NaN,NaN,,Digitally Downloaded,"It may sound like I’m being harsh on Ginger, b...",50,2016-11-21 00:00:00,0,0,c
5,2.0,NaN,NaN,,God is a Geek,JoJo’s Bizarre Adventure: Eyes in Heaven featu...,85,2016-07-04 00:00:00,0,0,c
6,2.0,NaN,NaN,,Gaming Age,"All in all, JoJo\s Bizarre Adventure: Eyes of ...",83,2016-07-06 00:00:00,0,0,c
7,2.0,NaN,NaN,,GameCritics,While I’m not familiar with the manga this gam...,75,2016-08-07 00:00:00,0,0,c
8,2.0,NaN,NaN,,IGN Spain,Fun and frantic but not as deep or as beautifu...,75,2016-07-11 00:00:00,0,0,c
9,2.0,NaN,NaN,,Atomix,"It tries to innovate, but it\s a game aimed fo...",75,2016-07-05 00:00:00,0,0,c


In [329]:
# Combining both summary and critic reviews doc2vec :model_summary and model 
summary_critics_docvecs = np.zeros(shape=(len(model_summary.docvecs),400))
for i in range(len(model_summary.docvecs)):
    summary_critics_docvecs[i] = np.concatenate((model_summary.docvecs[i],model.docvecs[i]), axis = 0)
#save pickle file


In [321]:
summary_critics_docvecs = np.concatenate((model_summary.docvecs,model.docvecs), axis = 0)

# save as a pickle
# load pickel file

# recommendation function 
len(np.concatenate((model_summary.docvecs[0],model.docvecs[0]), axis = 0))

400

In [331]:
import pickle
with open('summary_critics_docvecs.pickle', 'wb') as handle:
    pickle.dump(summary_critics_docvecs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [334]:
# For the app, the code for the recommendation would load a pickle and produce a recommendation 

#to load a pickle file
with open('summary_critics_docvecs.pickle', 'rb') as f:
    summary_critics_docvecs = pickle.load(f)

In [368]:
# Auxiliary functions for simple recommendation system 

# Calculate cosine similarity between two vecotrs 
def cossim(v1, v2): 
    return np.dot(v1, v2) / np.sqrt(np.dot(v1, v1)) / np.sqrt(np.dot(v2, v2)) 

# return top_n values from a list
def top_n_index(l,n):
    return sorted(range(len(l)), key=lambda i: l[i])[-(n+1):-1] #-1 to take off the own product from the returned index list

# return a dataframe with top_n products with cosine similarity score
def recommend(game_index, top_n, products_dataset, inputs):
    input_vec = inputs[products_dataset.loc[products_dataset['gameID']==game_index].index[0]]
    
    #compute similarity array
    sim_array = map(lambda v: cossim(input_vec, v), inputs)
    
    recommendation_index = top_n_index(sim_array, top_n)
    
    # return list of products 
    top_products = copy.deepcopy(products_dataset.iloc[recommendation_index,][['gameID']])
    top_products['index']= top_products.index
    
     # initialize an empty column for cosine similarity
    top_products['cossim']=0
    
    for i in range(len(index)):
        # note - this results in a bug: top_products['cossim'].loc[index[i],] = sim_array[index[i]]
        top_products.loc[recommendation_index[i],'cossim'] = sim_array[recommendation_index[i]]
        
    # merge game name
    top_products['gameID'] = top_products.apply(lambda row: int(row['gameID']),axis = 1)
    top_products = pd.merge(top_products, games[['gameID','name']], on='gameID')
    
    return top_products

In [378]:
gameID = 397
print games[games['gameID']==gameID]['name']
recommend(gameID, 5, games, summary_critics_docvecs )

396    Mario Tennis: Ultra Smash
Name: name, dtype: object


,gameID,index,cossim,name
0,12082,12081,0.722516,Mario Tennis Open
1,5678,5677,0.725441,Mario & Sonic at the London 2012 Olympic Games
2,11852,11851,0.746541,Super Smash Bros. for Wii U
3,18761,18760,0.748116,Mario Tennis: Power Tour
4,7792,7791,0.760362,Virtua Tennis 4
